# 🇵🇭 Philippine Elections OSINT Tool (Beta)
This tool helps Filipino citizens gather open-source information about political candidates.

In [ ]:
!pip install requests beautifulsoup4 googlesearch-python newspaper3k transformers lxml_html_clean --quiet

In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from newspaper import Article
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
# Load summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
def get_news_links(query, num=5):
    links = list(search(f"{query} site:news.abs-cbn.com OR site:philstar.com OR site:rappler.com", num_results=num))
    return links

def extract_and_summarize(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        text = article.text[:2000]
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        return f"⚠️ Failed to process article: {e}"

def run_osint_tool(candidate):
    display(Markdown(f"## 🔍 Searching for: **{candidate}**"))
    links = get_news_links(candidate)
    if not links:
        display(Markdown("❌ No articles found. Try a different name."))
        return
    for link in links:
        display(Markdown(f"### 🔗 [News Article]({link})"))
        summary = extract_and_summarize(link)
        display(Markdown(f"**📝 Summary:** {summary}"))

In [ ]:
candidate_input = widgets.Text(
    value='Juan Dela Cruz',
    placeholder='Enter candidate name',
    description='Candidate:',
    disabled=False
)

search_button = widgets.Button(description="Search Info 🔎")

def on_button_clicked(b):
    run_osint_tool(candidate_input.value)

search_button.on_click(on_button_clicked)

display(candidate_input, search_button)